# Visualize Parity between BMTK and BrainLGN-X

This notebook compares outputs from BMTK's LNUnit and BrainLGN-X's LGNNeuron on the same stimulus and filter stack.
It overlays rate time series, plots residuals, and reports numerical metrics (MAE/RMSE/MaxAbs/Corr).

In [ ]:
import os, sys, numpy as np, matplotlib.pyplot as plt

# Ensure package root on path (adjust if running from elsewhere)
PKG_ROOT = os.path.abspath(os.path.join('..'))
if PKG_ROOT not in sys.path: sys.path.insert(0, PKG_ROOT)

from brainlgn_x.neuron import LGNNeuron
from brainlgn_x.filters import GaussianSpatialFilter, TemporalFilterCosineBump, SpatioTemporalFilter
from brainlgn_x.transfer import ScalarTransferFunction

from bmtk.simulator.filternet.lgnmodel.lnunit import LNUnit
from bmtk.simulator.filternet.lgnmodel.movie import Movie

In [ ]:
def build_components(amplitude=1.0, bias=0.0):
    spatial = GaussianSpatialFilter(translate=(0.0, 0.0), sigma=(2.0, 2.0))
    tfilt   = TemporalFilterCosineBump(weights=(0.4, -0.3), kpeaks=(20.0, 60.0), delays=(0, 0))
    st      = SpatioTemporalFilter(spatial, tfilt, amplitude=float(amplitude))
    # Use Max to avoid two-arg Heaviside signature issues
    xfer    = ScalarTransferFunction(f'Max(0, s + {bias})')
    return st, xfer

## Generate stimulus and compute outputs (separable path)

In [ ]:
# Parameters
rng = np.random.RandomState(0)
T, H, W = 1000, 16, 16
frame_rate = 1000.0  # Hz
stim = rng.randn(T, H, W) * 0.05  # random stimulus

# Build filters
st, xfer = build_components(amplitude=1.0, bias=0.0)

# BMTK reference (separable)
movie  = Movie(stim, frame_rate=frame_rate)
ln_ref = LNUnit(st, xfer)
t_ref, y_ref = ln_ref.get_cursor(movie, separable=True).evaluate()
y_ref = np.array(y_ref)

# BrainLGN-X
neuron = LGNNeuron(spatial_filter=st.spatial_filter, temporal_filter=st.temporal_filter,
                   transfer_function=xfer, amplitude=st.amplitude)
y_new = neuron.evaluate(stim, separable=True, frame_rate=frame_rate)

# Metrics
diff = y_new - y_ref
mae  = np.mean(np.abs(diff))
rmse = np.sqrt(np.mean(diff**2))
mx   = np.max(np.abs(diff))
corr = np.corrcoef(y_ref, y_new)[0,1]
print(f'MAE={mae:.3e}, RMSE={rmse:.3e}, MaxAbs={mx:.3e}, Corr={corr:.6f}')

## Plots: overlay, residual, scatter, histogram

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(11, 6))
axs = axs.ravel()

axs[0].plot(y_ref, label='BMTK', lw=1)
axs[0].plot(y_new, label='BrainLGN-X', lw=1, alpha=0.7)
axs[0].set_title('Rates overlay'); axs[0].legend()

axs[1].plot(diff, lw=1, color='C3')
axs[1].axhline(0, color='k', lw=0.5)
axs[1].set_title('Residual (new - ref)')

axs[2].scatter(y_ref, y_new, s=6, alpha=0.6)
m = float(max(y_ref.max(), y_new.max()))
axs[2].plot([0, m], [0, m], 'k--', lw=1)
axs[2].set_xlabel('ref'); axs[2].set_ylabel('new'); axs[2].set_title('Scatter')

axs[3].hist(diff, bins=50, color='C1', alpha=0.85)
axs[3].set_title('Residual histogram')

plt.tight_layout(); plt.show()

## Optional: non-separable path comparison

In [ ]:
movie_ns  = Movie(stim, frame_rate=frame_rate)
ln_ref_ns = LNUnit(st, xfer)
t_ns, y_ref_ns = ln_ref_ns.get_cursor(movie_ns, separable=False, threshold=0.0).evaluate(downsample=1)
y_ref_ns = np.array(y_ref_ns)

y_new_ns = neuron.evaluate(stim, separable=False, threshold=0.0, downsample=1, frame_rate=frame_rate)
diff_ns  = y_new_ns - y_ref_ns
print('Non-separable: MAE=%.3e, RMSE=%.3e, MaxAbs=%.3e' % (
    np.mean(np.abs(diff_ns)), np.sqrt(np.mean(diff_ns**2)), np.max(np.abs(diff_ns))
))

plt.figure(figsize=(10,3))
plt.plot(y_ref_ns, label='BMTK (ns)', lw=1)
plt.plot(y_new_ns, label='BrainLGN-X (ns)', lw=1, alpha=0.7)
plt.legend(); plt.title('Non-separable overlay'); plt.show()